In [114]:
import pandas as pd 
import numpy as np

In [775]:
df = pd.read_csv('agaricus-lepiota.data',names = ["Class","cap-shape","cap-surface","cap-color",
                                                  "bruises","odor","gill-attachment","gill-spacing",
                                                  "gill-size","gill-color","stalk-shape","stalk-root",
                                                  "stalk-surface-above-ring","stalk-surface-below-ring",
                                                  "stalk-color-above-ring","stalk-color-below-ring",
                                                  "veil-type","veil-color","ring-number","ring-type",
                                                  "spore-print-color","population","habitat"])

In [786]:
df.dropna(axis=0,inplace=True)

In [787]:
df = df[df['stalk-root'] != '?']
df.index = range(0, df.shape[0])
df

,Class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5639,e,b,y,n,f,n,f,c,b,w,...,y,n,n,p,w,t,p,w,y,p
5640,e,x,y,n,f,n,f,c,b,w,...,y,n,n,p,w,t,p,w,y,p
5641,e,x,y,g,t,n,f,c,b,w,...,s,w,w,p,w,t,p,w,y,p
5642,p,x,y,c,f,m,f,c,b,y,...,y,c,c,p,w,n,n,w,c,d


## Q2(a)

In [796]:
def info_gain(sub_df):
    p_ep = sub_df.iloc[:,0]
    c_ep = sub_df.iloc[:,1]
    count = p_ep.count()
    each_count = p_ep.value_counts()
    each_count_p = each_count / count
    ep_lst = []
    
    for n in c_ep.unique():
        lst = []
        for i in p_ep.unique():
            p = sub_df[(p_ep == i) & (c_ep == n)].count()[0]
            lst.append(p)
        arr = np.array(lst)
        arr_p = arr / arr.sum()
        if not np.isin(arr_p, 0).any():
            ep = -(np.log2(arr_p) * arr_p).sum()
            cond_ep = (c_ep.value_counts()[n] / count) * ep
            ep_lst.append(cond_ep)
        else:
            ep_lst.append(0)
     
    overall_ep = -(np.log2(each_count_p) * each_count_p).sum()
    return overall_ep - sum(ep_lst)


def highest_ig_column(df):
    current_ep = df.iloc[:,0]
    ig_lst = []
    for i in df.columns[1:]:
        ig = info_gain(df[[current_ep.name, i]])
        ig_lst.append(ig)
    arr = np.array(ig_lst)
    return df.columns[np.argmax(arr) + 1]


def whether_terminate(data,col):
    node = {}
    label = data.columns[0]
    label_value = data.iloc[:,0].unique()
    for i,n in data[col].value_counts().iteritems():
        label_list = []
        index_list = []
        varb_num_data = data[data[col] == i]
        for l in label_value:
            label_list.append(varb_num_data[varb_num_data[label] == l].count()[0])
            index_list.append(l)
        arr = np.array(label_list)
        if np.isin(arr, 0).any():
            pos = arr.argmax()
            node[i] = index_list[pos]
            data.drop(data[data[col] != i].index, axis = 0)
        else:
            node[i] = 'unterminated'
    return node, data

def build_decision_tree(data, root, value):
    label_value = data.iloc[:, 0].unique()
    label_list = []
    label_dict = {}
    next_root = {}
    for i in label_value:
        label_list.append(i)
    label_arr = np.array(label_list)

    if value.isalpha():
        if data.count()[0]:
            col = highest_ig_column(data)
            sub_tree, sub_data = whether_terminate(data, col)
            root[value] = label_dict
            root[value][col] = sub_tree
            next_root = root[value][col]
    else:
        if data.count()[0]:
            col = highest_ig_column(data)
            sub_tree, sub_data = whether_terminate(data, col)
            root[col] = sub_tree
            next_root = root[col]
    for i, n in list(next_root.items()):
        if not np.isin(n, label_arr):
            build_decision_tree(sub_data[sub_data[col] == i], next_root, i)

def decision_tree(data,value = ''):
    decision_tree = {}
    build_decision_tree(data, decision_tree, value)
    return decision_tree

print(decision_tree(training_df))

{'odor': {'n': {'spore-print-color': {'k': 'e', 'n': 'e', 'w': {'cap-color': {'n': 'e', 'c': 'e', 'g': 'e', 'p': 'e', 'y': 'p', 'w': 'p'}}, 'r': 'p'}}, 'f': 'p', 'l': 'e', 'a': 'e', 'p': 'p', 'c': 'p', 'm': 'p'}}


## Q2(c):

In [631]:
from sklearn.model_selection import train_test_split 

In [789]:
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:,1:], df.iloc[:,0])

In [807]:
training_df = x_train.copy()
training_df.insert(0,'Class', y_train)
training_df

,Class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
2984,e,x,f,e,t,n,f,c,b,u,...,s,w,g,p,w,o,p,k,y,d
3860,e,x,y,n,t,n,f,c,b,n,...,s,w,w,p,w,o,p,n,v,d
3961,e,f,f,g,t,n,f,c,b,w,...,s,p,p,p,w,o,p,k,v,d
4566,p,x,f,y,f,f,f,c,b,h,...,k,b,p,p,w,o,l,h,v,d
5099,p,f,y,y,f,f,f,c,b,g,...,k,p,p,p,w,o,l,h,v,d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5436,p,x,s,g,t,f,f,c,b,w,...,s,w,w,p,w,o,p,h,s,g
4534,p,x,y,y,f,f,f,c,b,h,...,k,n,p,p,w,o,l,h,y,p
2449,e,f,f,g,t,n,f,c,b,n,...,s,w,w,p,w,o,p,k,y,d
1852,e,x,f,e,t,n,f,c,b,w,...,s,w,g,p,w,o,p,n,v,d


## Q2(d)

In [808]:
from sklearn.metrics import confusion_matrix


def predict_label(decision_tree, single_test_data, parent_node = ''):
    keys = decision_tree.keys()
    keys_lst = list(keys)
    if len(keys_lst) == 1:
        return predict_label(decision_tree[keys_lst[0]], single_test_data, keys_lst[0])
    label_value = single_test_data[parent_node]
    if not isinstance(decision_tree[label_value], dict):
        return decision_tree[label_value]
    return predict_label(decision_tree[label_value], single_test_data, keys)

def accuracy(decision_tree, test_data, test_data_label, train_data, train_data_label):
    pred_label_lst = []
    verify_label_lst = []
    correct = 0
    verify = 0
    total_row = test_data.shape[0]
    total_train_row = train_data.shape[0]
    
    for i in range(total_row):
        each_instance = test_data.iloc[i]
        pred_label = predict_label(decision_tree,each_instance)
        pred_label_lst.append(pred_label)
        if pred_label == test_data_label.iloc[i]:
            correct += 1
            
    for i in range(total_train_row):
        verify_instance = train_data.iloc[i]
        verify_label = predict_label(decision_tree,verify_instance)
        verify_label_lst.append(verify_label)
        if verify_label == train_data_label.iloc[i]:
            verify += 1
        
    return correct / total_row, pred_label_lst, verify / total_train_row, verify_label_lst

my_decision_tree = decision_tree(training_df)
accuracy, pred_label_lst, train_error, verify_label_lst = accuracy(my_decision_tree, x_test, y_test, x_train, y_train)
cm = confusion_matrix(y_test, pred_label_lst)
cm_train = confusion_matrix(y_train, verify_label_lst)
print(f'accuracy socre for test data:  {accuracy}', f'{cm}   confusion matrix for test data', sep = '\n\n')
print()
print()
print(f'accuracy socre for train data:  {train_error}', f'{cm_train}   confusion matrix for train data', sep = '\n\n')

accuracy socre for test data:  1.0

[[875   0]
 [  0 536]]   confusion matrix for test data


accuracy socre for train data:  1.0

[[2613    0]
 [   0 1620]]   confusion matrix for train data
